Playing ATARI-ram games using python-neat

# Import

In [17]:
import numpy as np
import gym
from gym import wrappers
from __future__ import print_function
import os
import neat
import visualize

# Define config

In [18]:
fc_config_filename = 'fc.config'
game = 'MsPacman-ram-v0'
num_evaluations = 3
num_cores = 3

# Create environment

In [19]:
env = gym.make(game)

[2017-02-07 15:52:57,462] Making new env: MsPacman-ram-v0


# Create neat-python population

In [20]:
# Load configuration.
config_initial = neat.Config(neat.DefaultGenome, neat.DefaultReproduction,
                     neat.DefaultSpeciesSet, neat.DefaultStagnation, fc_config_filename)

config_initial.genome_config.num_inputs = env.observation_space.shape[0] + 1
config_initial.genome_config.num_outputs = env.action_space.n + 1

In [21]:
game_fc_config_filename = 'fc-' + game + '.config'

config_initial.save(game_fc_config_filename)

config = neat.Config(neat.DefaultGenome, neat.DefaultReproduction,
                     neat.DefaultSpeciesSet, neat.DefaultStagnation, game_fc_config_filename)

# Create the population, which is the top-level object for a NEAT run.
p = neat.Population(config)

In [22]:
config.species_set_config

{'compatibility_threshold': 3.0}

In [23]:
config.save('aa')

# Add reporters

In [13]:
# Add a stdout reporter to show progress in the terminal.
p.add_reporter(neat.StdOutReporter())
stats = neat.StatisticsReporter()
p.add_reporter(stats)
p.add_reporter(neat.Checkpointer(5))

# Define fitness via game score

In [15]:
# a = argmax_a Q(s,a)
def predict_action(observation, network):
    observation = observation / 255. * 2 - 1
    output = network.activate(observation)
    action = np.argmax(output)
    return(action)

# play num_evaluations games, take mean
def evaluate_network(env, network):
    rewards = []
    i = 0
    while i < num_evaluations:
        rewards += [get_reward(env, network)]
        i += 1

    res = np.array(rewards).mean()
    return res

# play 1 game with network
def get_reward(env, network):
    observation = env.reset()
    done = False
    iteration, total_reward = 0, 0
        
    while not done:
        #env.render()
        action = predict_action(observation, network)
        observation, reward, done, info = env.step(action)
        total_reward += reward

        #if iteration % 500 == 0:
        #    print(str(iteration))

        #if total_reward < -12000 or iteration >= 8000:
        #    break

        iteration += 1
        
    return total_reward

def evaluate_genome(genome, config):
    network = neat.nn.FeedForwardNetwork.create(genome, config)
    fitness = evaluate_network(env, network)
    return fitness
    
#print(evaluate_genome(p.species.get_species(1).members[1], config))

evaluator = neat.parallel.ParallelEvaluator(num_workers = num_cores, eval_function = evaluate_genome, timeout = None)

NameError: name 'num_cores' is not defined

# Run evolution

In [16]:
# Run evolution
winner = p.run(evaluator.evaluate, 1)

NameError: name 'evaluator' is not defined

# Print results

In [6]:
# Display the winning genome.
print('\nBest genome:\n{!s}'.format(winner))

# Show output of the most fit genome against training data.
print('\nOutput:')
winner_net = neat.nn.FeedForwardNetwork.create(winner, config)
for xi, xo in zip(xor_inputs, xor_outputs):
    output = winner_net.activate(xi)
    print("input {!r}, expected output {!r}, got {!r}".format(xi, xo, output))

#node_names = {-1:'A', -2: 'B', 0:'A XOR B'}
#visualize.draw_net(config, winner, True, node_names=node_names)
#visualize.plot_stats(stats, ylog=False, view=True)
#visualize.plot_species(stats, view=True)

#p = neat.Checkpointer.restore_checkpoint('neat-checkpoint-4')
#p.run(eval_genomes, 10)

NameError: name 'winner' is not defined